# Carvana exploration notebook

Import libraries

In [9]:
# import libraries
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K

import os
import itertools
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


Get the file name IDs and split into train and validation sets

In [17]:
# define directories
dataset_dir = '/home/ubuntu/carvana/input/data/'
image_dir = dataset_dir + 'images'
mask_dir = dataset_dir + 'masks'

In [23]:
img_width = 128
img_height = 128
batch_size=16

In [22]:
# Define data augmentations for training set
data_gen_args = dict(rescale=1./255,
                     shear_range=0.1,
                     rotation_range=4,
                     zoom_range=0.1,
                     horizontal_flip=True,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     validation_split=0.2) # 20% validation set

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

seed = 42

# Create generator for training images
train_image_generator = image_datagen.flow_from_directory(
    image_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    seed=seed,
    subset='training')

# Create generator for training masks
train_mask_generator = mask_datagen.flow_from_directory(
    mask_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    seed=seed,
    subset='training')

# Create generator for validation images
val_image_generator = image_datagen.flow_from_directory(
    image_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    seed=seed,
    subset='validation')

# Create generator for validation masks
val_mask_generator = mask_datagen.flow_from_directory(
    mask_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    seed=seed,
    subset='validation')

# Combine generators into single training and validation generators for model training
train_generator = zip(train_image_generator, train_mask_generator)
validation_generator = zip(val_image_generator, val_mask_generator)

Found 4071 images belonging to 1 classes.
Found 4071 images belonging to 1 classes.
Found 1017 images belonging to 1 classes.
Found 1017 images belonging to 1 classes.


In [ ]:
def dice_coeff(true, pred):
    smooth = 1.
    true_flat = K.flatten(true)
    pred_flat = K.flatten(pred)
    intersection = K.sum(true_flat * pred_flat)
    score = (2. * intersection + smooth) / (K.sum(true_flat) + K.sum(pred_flat) + smooth)
    return score

def dice_loss(true, pred):
    loss = 1 - dice_coeff(true, pred)
    return loss